# Extração de passagens aéreas da empresa GOL

> Esse notebook contém uma automação para extrair o preço das passagens aéreas do site da empresa GOL. Nesse sentido, ela retornorá 3 arquivos XLSX, sendo o primeiro com os preços de ida e volta; o segundo, somente ida; e o terceiro, somente de volta.

## Bibliotecas usadas nessa automação.

In [2]:
import os
import time
from datetime import date
from dateutil.relativedelta import relativedelta

from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, InvalidSessionIdException, \
    StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

## Classe Browser
>Ela contém métodos e atributos para manipular o navegador usado na Automação. Nesse sentido, ela é baseada na biblioteca Sellenium

In [3]:
class Browser:
    def __init__(self):
        self.servico = Service(ChromeDriverManager().install())
        self._driver = self.retornar_browser()

    def retornar_browser(self):
        return webdriver.Chrome(service=self.servico)

    def acessar_site(self, link):
        self._driver.get(link)

    def encontre_by_xpath(self, xpath):
        elemento = self._driver.find_element(By.XPATH, xpath)
        return elemento

    def encontre_elements_by_class(self, class_name):
        elementos = self._driver.find_elements(By.CLASS_NAME, class_name)
        return elementos

    @staticmethod
    def click_elemento(elemento):
        elemento.click()

    @staticmethod
    def send_keys(elemento, string):
        elemento.send_keys(string)

    def click_home_key(self):
        self._driver.find_element(By.XPATH, '//body').send_keys(Keys.HOME)

    def retorna_html_page(self):
        return self._driver.page_source

    def retornar_todas_as_abas_abertas(self):
        return self._driver.window_handles

    def mudar_aba_browser(self, numero_aba):
        abas = self.retornar_todas_as_abas_abertas()
        self._driver.switch_to.window(abas[numero_aba])

    @staticmethod
    def send_enter_key(elemento):
        elemento.send_keys(Keys.ENTER)

    def click_end_key(self):
        self._driver.find_element(By.XPATH, '//body').send_keys(Keys.END)

    def find_element_xpath_explicity_wait_visibility(self, tempo, xpath):
        elemento = WebDriverWait(self._driver, tempo).until(EC.visibility_of_element_located((By.XPATH, xpath)))
        return elemento

    def find_element_class_name_explicity_wait_visibility(self, tempo, class_name):
        elemento = WebDriverWait(self._driver, tempo).until(EC.visibility_of_element_located((By.CLASS_NAME, class_name)))
        return elemento

    def find_element_xpath_explicity_wait_clickable(self, tempo, xpath):
        element = WebDriverWait(self._driver, tempo).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        return element

    def wait_url_matches(self, pattern):
        wait = WebDriverWait(self._driver,10)
        wait.until(EC.url_matches(pattern))

    def get_actual_url(self):
        url = self._driver.current_url
        return url

## Classe Itinerário

> Ela contém a lista de aeroportos que devem ser analisados. Nesse sentido, ela irá retornar os aeroportos de origem e destino para a automação.

In [4]:
# Clase que contém o itinerário de voos
class Itinerario:
    def __init__(self):
        self._lista_aeroporto = self.lista_aeroporto()

    # Retornar o itinerario a ser usado pelo browser
    def retorna_itinerario(self):
        for ida in self._lista_aeroporto:
            for volta in self._lista_aeroporto:
                if ida == volta:
                    pass
                else:
                    yield ida, volta

    # Método que retornar a lista com os códigos dos aeroportos
    @staticmethod
    def lista_aeroporto():
        lista_aeroporto = [
            "FORTALEZA",
            "RBR",
            "GIG",
            "SALVADOR",
            "POA",
            "BELEM",
            "REC",
            "GRU",
            "CWB",
            "BSB",
            "CNF",
            "VIX",
            "SLZ",
            "AJU",
            "CGR",
            "GYN",
        ]
        return lista_aeroporto

## Classe GOL

> Ela que contém o método de extração em si. Por isso, para a executar, a classe GOL herda as outras duas classes (Itinerário e Browser)

In [5]:
# Classe que contém as ações no site da GOL
class Gol(Browser, Itinerario):
    def __init__(self):
        # self._driver = self.retornar_browser(servico)
        # Link utilizado para acessar site da GOL
        self._url = 'https://www.voegol.com.br/'

        # Data atual
        self._data_atual = self.calcula_data_atual()

        # Data de partida
        self._data_ida = self.calcula_data_ida()

        # Data de retorno
        self._data_volta = self.calcula_data_volta()

        # Data atual formatada,sendo o padrão (00/00/0000)
        self.data_atual_formatada = self._data_atual.strftime("%d/%m/%Y")

        # Data de partida formatada, sendo o padrão (00/00/0000)
        self._data_ida_formatada = self._data_ida.strftime("%d/%m/%Y")

        # Data de retorno formatada, sendo o padrão (00/00/0000)
        self._data_volta_formatada = self._data_volta.strftime("%d/%m/%Y")

        # Esse atributo guarda os nomes das colunas utilizadas pelo DataFrame
        self.coluna_nomes = pd.MultiIndex.from_tuples([('ORIGEM', f'Data Partida: {self._data_ida_formatada}'), ('DESTINO', f'Data Retorno: {self._data_volta_formatada}'), ('PREÇO DA PASSAGEM', f'Dia Coleta: {self.data_atual_formatada}')])
        
        # Df que guarda os preços dos itinerarios somente de partida
        self.df_ida_origem = pd.DataFrame(columns=self.coluna_nomes)

        # Df que guarda os preços dos itinerarios somente de retorno
        self.df_volta_origem = pd.DataFrame(columns=self.coluna_nomes)

        # Df que guarda os itinerários de ida e volta
        self.df_preco_ida_volta = pd.DataFrame(columns=self.coluna_nomes)
        
        # Para instanciar super class Itineario
        Itinerario.__init__(self)

    # Método que realiza os processos do scraping
    def scraping(self):
        # Var que guarda o tempo de início da emtração
        comeco = time.time()

        print(self.data_atual_formatada)
    
        # Para criar o arquivo que guardará os preços coletados, o qual, em seu nome, contém o nome
        data_atual = self._data_atual.strftime("%d_%m_%Y")
        try:
            os.mkdir(f'Extração_GOL_{data_atual}')
        except FileExistsError:
            pass

        for ida, volta in self.retorna_itinerario():
            while True:
                try:
                    print(f'Itineário: {ida} para {volta}.')
                    Browser.__init__(self)
                    # Acessar site
                    self.acessar_site(self._url)

                    # Para aceitar cookies do site da Gol
                    try:
                        xpath = '//*[@id = "onetrust-accept-btn-handler"]'
                        element_cookie = self.find_element_xpath_explicity_wait_visibility(20, xpath)
                        element_cookie.click()
                        print('Cookie aceito')

                    except TimeoutException:
                        print('Cookie não aceito')

                    # Clicar aeroporto ida
                    xpath = '//*[@id="edit-fieldset-origin"]'
                    tempo = 15
                    elemento_ida = self.find_element_xpath_explicity_wait_visibility(tempo, xpath)
                    self.click_elemento(elemento_ida)

                    # Escrever destino de volta
                    xpath = '//*[@id="edit-fieldset-origin"]/div/span[1]/span[1]/span[1]/input'
                    elemento_recebe_origem = self.find_element_xpath_explicity_wait_visibility(5, xpath)
                    self.send_keys(elemento_recebe_origem, ida)
                    # time.sleep(0.5)
                    self.send_enter_key(elemento_recebe_origem)

                    # Escrever destino do voo
                    xpath = '//*[@id="edit-fieldset-destiny"]'
                    elemento_volta = self.find_element_xpath_explicity_wait_visibility(tempo, xpath)
                    self.click_elemento(elemento_volta)

                    # Escrever itinerario de voo volta
                    xpath = '//*[@id="edit-fieldset-destiny"]/div[1]/span[1]/span[1]/span[1]/input'
                    elemento_input_volta = self.find_element_xpath_explicity_wait_visibility(5, xpath)
                    self.send_keys(elemento_input_volta, volta)
                    self.send_enter_key(elemento_input_volta)

                    # Clicar em calendário partida
                    xpath = '//*[@id="edit-fieldset-departure-date"]'
                    elemento_volta = self.find_element_xpath_explicity_wait_clickable(5, xpath)
                    elemento_volta.click()

                    # Escrever data partida
                    xpath = '//*[@id="edit-departure-date"]'
                    elemento_data_ida_escrever = self.find_element_xpath_explicity_wait_visibility(5, xpath)
                    data_partida = self._data_ida.strftime("%d%m%Y")
                    self.send_keys(elemento_data_ida_escrever, data_partida)
                    # time.sleep(0.4)

                    # Para selecionar a data escolhida
                    xpath = '//*[@id="ticket-search"]/form/div[2]/div/div[2]/button'
                    elemento_but_selecionr_data_ida_escolhida = self.find_element_xpath_explicity_wait_clickable(5,
                                                                                                                 xpath)
                    self.click_elemento(elemento_but_selecionr_data_ida_escolhida)

                    # Para selecionar field com data de retorno
                    xpath = '//*[@id="edit-fieldset-back-date"]'
                    elemento_fild_data_retorno = self.find_element_xpath_explicity_wait_visibility(5, xpath)
                    self.click_elemento(elemento_fild_data_retorno)
                    time.sleep(0.4)

                    # Para escrever data selecionada de retorno
                    xpath = '//*[@id="edit-back-date"]'
                    elemento_input_data_retorno = self.find_element_xpath_explicity_wait_visibility(5, xpath)
                    data_volta = self._data_volta.strftime("%d%m%Y")
                    self.send_keys(elemento_input_data_retorno, data_volta)

                    # Para selecionatr as data de retorno
                    xpath = '//*[@id="ticket-search"]/form/div[2]/div/div[2]/button'
                    elemento_but_selecionar_data_retorno = self.find_element_xpath_explicity_wait_clickable(5, xpath)
                    self.click_elemento(elemento_but_selecionar_data_retorno)

                    # Para clicar no botão de buscar o itinerario
                    xpath = '//*[@id="ticket-search-submit"]'
                    elemento_but_search_rota = self.find_element_xpath_explicity_wait_clickable(5, xpath)
                    self.click_elemento(elemento_but_search_rota)

                    # DataFrames que serão utilizados caso não haja passagens
                    df_sem_passagem_ida = self.cria_df_sem_passagem(ida, volta)
                    df_sem_passagem_volta = self.cria_df_sem_passagem(volta, ida)

                    # Verificar se há passagens para essa rota
                    xpath = '/html/body/app-root/b2c-flow/main/b2c-select-flight/div/section/form/div[1]'  # XPATH do Primeiro voo
                    try:
                        time.sleep(13)
                        url_ida = 'https://b2c.voegol.com.br/compra/selecao-de-voo/ida'
                        url_atual = self._driver.current_url
                        pagina_preco_ida = url_atual == url_ida
                        if pagina_preco_ida:
                            primeiro_voo = self.find_element_xpath_explicity_wait_visibility(10, xpath)

                            print('Há voos para essa rota')
                            price_final_ida = self.realizar_extracao_precos()
                            self.click_end_key()
                            time.sleep(1)
                            for i in range(len(price_final_ida)):
                                df_ida_juntar = pd.DataFrame(data=[ida, volta, price_final_ida[i]]).T
                                df_ida_juntar.columns = self.coluna_nomes
                                self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_ida_juntar], axis=0)
                                self.df_ida_origem = pd.concat([self.df_ida_origem, df_ida_juntar], axis=0)

                            print(price_final_ida)
                            self._driver.execute_script("arguments[0].scrollIntoView();", primeiro_voo)
                            time.sleep(3)
                            self.click_elemento(primeiro_voo)

                            # Clicar em pesquisar o voo de volta
                            xpath = '/html/body/app-root/b2c-flow/main/b2c-select-flight/div/section/form/b2c-anchor-bar/div/div/button'
                            element_but_pesquisa_volta = self.find_element_xpath_explicity_wait_visibility(2, xpath)
                            self.click_elemento(element_but_pesquisa_volta)

                            try:
                                time.sleep(13)
                                url_volta = 'https://b2c.voegol.com.br/compra/selecao-de-voo/volta'
                                pagina_preco_retorno = url_volta == self._driver.current_url

                                if pagina_preco_retorno:
                                    xpath = '/html/body/app-root/b2c-flow/main/b2c-select-flight/div/section/form/div[1]'
                                    primeiro_voo_volta = self.find_element_xpath_explicity_wait_visibility(10, xpath)
                                    self.click_end_key()
                                    time.sleep(0.65)

                                    print('Há voos de volta!')
                                    price_final_volta = self.realizar_extracao_precos()
                                    print(price_final_volta)
                                    for i in range(len(price_final_volta)):
                                        df_volta_juntar = pd.DataFrame(data=[volta, ida, price_final_volta[i]]).T
                                        df_volta_juntar.columns = self.coluna_nomes
                                        self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_volta_juntar], axis=0)
                                        self.df_volta_origem = pd.concat([self.df_volta_origem, df_volta_juntar], axis=0)

                                    print(self.df_preco_ida_volta)
                                    print(self.df_ida_origem)
                                    print(self.df_volta_origem)
                                    
                                    self._driver.quit()
                                    break

                                else:
                                    pass

                            except TimeoutException:
                                print('Não há voos de volta!')
                                self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_sem_passagem_volta], axis=0)
                                self.df_volta_origem = pd.concat([self.df_volta_origem, df_sem_passagem_volta], axis=0)
                                self._driver.quit()
                                break

                        else:
                            pass

                    except TimeoutException:
                        print('Não há voos para essa rota')
                        self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_sem_passagem_ida], axis=0)
                        self.df_ida_origem = pd.concat([self.df_ida_origem, df_sem_passagem_ida], axis=0)
                        self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_sem_passagem_volta], axis=0)
                        self.df_volta_origem = pd.concat([self.df_volta_origem, df_sem_passagem_volta], axis=0)
                        self._driver.quit()
                        break

                except RuntimeError:
                    print(f'Erro: RuntimeError. Vai repetir {ida} de {volta}!')
                    self._driver.quit()
                    pass

                except TimeoutException:
                    print(f'Erro: TimeoutException. Vai repetir {ida} de {volta}!')
                    self._driver.quit()
                    pass

                except ElementClickInterceptedException:
                    self._driver.quit()
                    print(f'Erro: ElementClickInterceptedException. Vai repetir {ida} de {volta}')

                except InvalidSessionIdException:
                    self._driver.quit()
                    print(f'Erro: InvalidSessionIdException. Vai repetir {ida} de {volta}')

                except StaleElementReferenceException:
                    print(f'Erro: Stale element reference. Vai repetir {ida} de {volta}!')

            self.df_preco_ida_volta.to_excel(f'Extração_GOL_{data_atual}/df_preco_gol_ida_volta_{data_atual}.xlsx')
            self.df_ida_origem.to_excel(f'Extração_GOL_{data_atual}/df_preco_gol_ida_origem_{data_atual}.xlsx')
            self.df_volta_origem.to_excel(f'Extração_GOL_{data_atual}/df_preco_gol_volta_destino_{data_atual}.xlsx')

        final = time.time()
        print(final - comeco)

    def cria_df_sem_passagem(self, aero1, aero2):
        data_sem_passagem = 'Sem voos disponíveis'
        df_sem_passagem = pd.DataFrame(data=[aero1, aero2, data_sem_passagem]).T
        df_sem_passagem.columns = self.coluna_nomes
        return df_sem_passagem

    def realizar_extracao_precos(self):
        html = self.retorna_html_page()
        soup = BeautifulSoup(html, 'html.parser')
        price = soup.findAll('span', {'class': 'a-desc__value a-desc__value--price'})
        price_final = []
        for preco in price:
            preco = preco.get_text().strip()
            price_final.append(preco)

        return price_final

    @staticmethod
    def calcula_data_atual():
        return date.today()

    def calcula_data_ida(self):
        data_ida = self._data_atual + relativedelta(months=2)
        return data_ida

    def calcula_data_volta(self):
        data_volta = self._data_ida + relativedelta(days=8)
        return data_volta

In [6]:
def scraping_gol():
    objeto_gol = Gol()
    objeto_gol.scraping()

In [7]:

if __name__ == '__main__':
    scraping_gol()

16/02/2023
Itineário: FORTALEZA para RBR.


[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 7.82MB/s]


Cookie aceito
Há voos para essa rota
['R$\xa01.131,71']
Há voos de volta!
['R$\xa01.114,37']
                    ORIGEM                  DESTINO      PREÇO DA PASSAGEM
  Data Partida: 16/04/2023 Data Retorno: 24/04/2023 Dia Coleta: 16/02/2023
0                FORTALEZA                      RBR            R$ 1.131,71
0                      RBR                FORTALEZA            R$ 1.114,37
                    ORIGEM                  DESTINO      PREÇO DA PASSAGEM
  Data Partida: 16/04/2023 Data Retorno: 24/04/2023 Dia Coleta: 16/02/2023
0                FORTALEZA                      RBR            R$ 1.131,71
                    ORIGEM                  DESTINO      PREÇO DA PASSAGEM
  Data Partida: 16/04/2023 Data Retorno: 24/04/2023 Dia Coleta: 16/02/2023
0                      RBR                FORTALEZA            R$ 1.114,37
Itineário: FORTALEZA para GIG.
Cookie aceito
Há voos para essa rota
['R$\xa0766,61', 'R$\xa0766,61', 'R$\xa0766,61', 'R$\xa0984,61', 'R$\xa0984,61', 'R$\xa09

WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: cannot determine loading status
from target frame detached
  (Session info: chrome=109.0.5414.122)
Stacktrace:
Backtrace:
	(No symbol) [0x005A6643]
	(No symbol) [0x0053BE21]
	(No symbol) [0x0043D960]
	(No symbol) [0x0042F218]
	(No symbol) [0x0042DC68]
	(No symbol) [0x0042E647]
	(No symbol) [0x00447958]
	(No symbol) [0x00440477]
	(No symbol) [0x0043FFAF]
	(No symbol) [0x00440723]
	(No symbol) [0x00438E85]
	(No symbol) [0x00438A9F]
	(No symbol) [0x00444956]
	(No symbol) [0x004481C6]
	(No symbol) [0x0042E9F1]
	(No symbol) [0x00444517]
	(No symbol) [0x004A7057]
	(No symbol) [0x0048FB76]
	(No symbol) [0x004649C1]
	(No symbol) [0x00465E5D]
	GetHandleVerifier [0x0081A142+2497106]
	GetHandleVerifier [0x008485D3+2686691]
	GetHandleVerifier [0x0084BB9C+2700460]
	GetHandleVerifier [0x00653B10+635936]
	(No symbol) [0x00544A1F]
	(No symbol) [0x0054A418]
	(No symbol) [0x0054A505]
	(No symbol) [0x0055508B]